In [1]:
import pandas as pd

In [3]:
titanic_csv = pd.read_csv('files/train.csv')
titanic_csv.head()

In [190]:
titanic_csv.shape

In [191]:
column: str = titanic_csv.columns[0] # Retorna cabecera de la columna
titanic_csv[column]

# Columns to omit:
- Name : Column 4
- PassengerId : column 1
- Fare : column 10
- Embarked : column 12
- Ticket : column 9

In [192]:
# Traspaso a un nuevo pandas.Dataframe omitiendo las columnas indicadas
titanic_df = titanic_csv.drop(columns=['Name', 'PassengerId', 'Fare', 'Embarked', 'Ticket'])

In [193]:
column_name = titanic_df.columns[6]
titanic_df.head()

In [194]:
# Dataframe.nunique(dropna=False) retorna conteo valores unicos incluyendo valores nan

# Columna Survived no tiene valores nan
titanic_df['Survived'] = titanic_df['Survived'].astype(float)

# Columna Pclass no tiene valores nan
titanic_df['Pclass'] = titanic_df['Pclass'].astype(float)

# Columna Sex no tiene valores nan
# Se pasan los valores male and female a numericos
titanic_df['Sex'] = titanic_df['Sex'].apply(lambda value: 1 if value == 'male' else 0)
titanic_df['Sex'] = titanic_df['Sex'].astype(float)

# Columna Age tiene valores nan
# Se rellena con valores 0 los nan
titanic_df['Age'] = titanic_df['Age'].fillna(0)
titanic_df['Age'] = titanic_df['Age'].astype(float)

# Columna SibSp no tiene valores nan
titanic_df['SibSp'] = titanic_df['SibSp'].astype(float)

# Columna Parch no tiene valores nan
titanic_df['Parch'] = titanic_df['Parch'].astype(float)

# Cabin tiene valores nan y se reemplazan por valores 0
titanic_df['Cabin'], unique = titanic_df['Cabin'].factorize(use_na_sentinel=True)
titanic_df['Cabin'] = titanic_df['Cabin'].replace(-1, 150)
titanic_df['Cabin'] = titanic_df['Cabin'].astype(float)

In [195]:
titanic_df['Cabin'].unique()

In [196]:
from sklearn.model_selection import train_test_split

# Separa valores en test y train
x_df = titanic_df.drop(columns=['Survived'])
y_df = titanic_df['Survived']

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=42)


In [197]:
# Escalar valores entre
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [198]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Creación de tensors

X_train_tensor = torch.tensor(data=X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(data=X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset_tensor = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset_tensor = TensorDataset(X_test_tensor, y_test_tensor)

# Creacion de DataLoader
batch_size = 32
train_loader = DataLoader(dataset=train_dataset_tensor, batch_size=batch_size)
test_loader = DataLoader(dataset=test_dataset_tensor, batch_size=batch_size)

In [267]:
import torch.nn as nn

# Creación del modelo
class TitanicModel(nn.Module):
    def __init__(self, input_size):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu( self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x




In [268]:
from torch import optim

input_size: int = X_train_tensor.shape[1]
titanic_model = TitanicModel(input_size=input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(titanic_model.parameters(), lr=0.001)


In [269]:
def train(model, train_loader, optimizer, criterion, epochs):

    loss_list_train = []
    iteration_list_train = []

    for epoch in range(epochs):
        model.train()
        epoch_loss_train = 0.0

        for index, (batch_x, batch_y) in enumerate(train_loader):
            optimizer.zero_grad()

            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)

            loss.backward()
            optimizer.step()

            epoch_loss_train += loss.item()

        loss_mean = epoch_loss_train / len(train_loader)
        loss_list_train.append(loss_mean)
        iteration_list_train.append(epoch + 1)

    return loss_list_train, iteration_list_train


In [270]:
def test(model, test_loader, criterion):
    model.eval()
    epoch_loss_test = 0.0
    loss_list_test = []
    iteration_list_test = []

    with torch.no_grad():
        for index, (batch_x, batch_y) in enumerate(test_loader):
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            epoch_loss_test += loss.item()

            print(f'Epoch {index + 1}/{len(test_loader)} Loss: {loss.item():.4f}')

            loss_list_test.append(loss.item())
            iteration_list_test.append(index + 1)


    loss_test_mean = epoch_loss_test / len(test_loader)
    return iteration_list_test, loss_list_test, loss_test_mean

In [271]:
epochs: int = 1000
loss_list_train, iteration_list_train = train(model=titanic_model, train_loader=train_loader, optimizer=optimizer, criterion=criterion, epochs=epochs)

In [272]:
import matplotlib.pyplot as plt

plt.plot(iteration_list_train, loss_list_train)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Curve TRAIN')
plt.show()

In [273]:
iteration_list_test, loss_list_test, loss_test_mean = test(model=titanic_model, test_loader=test_loader, criterion=criterion)

In [274]:
plt.plot(iteration_list_test, loss_list_test)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss Curve TEST')
plt.show()

In [229]:
# Testing for kaggle competition
test_csv = pd.read_csv('files/test.csv')
test_csv.head()

In [230]:
test_df = test_csv.drop(columns=['Name', 'PassengerId', 'Fare', 'Embarked', 'Ticket'])

test_df['Pclass'] = test_df['Pclass'].astype(float)

test_df['Sex'] = test_df['Sex'].apply(lambda value: 1 if value == 'male' else 0)
test_df['Sex'] = test_df['Sex'].astype(float)

test_df['Age'] = test_df['Age'].fillna(0)
test_df['Age'] = test_df['Age'].astype(float)

test_df['SibSp'] = test_df['SibSp'].astype(float)

test_df['Parch'] = test_df['Parch'].astype(float)

test_df['Cabin'], unique = test_df['Cabin'].factorize(use_na_sentinel=True)
test_df['Cabin'] = test_df['Cabin'].replace(-1, 150)
test_df['Cabin'] = test_df['Cabin'].astype(float)

In [ ]:
# Columna Survived no tiene valores nan


# Columna Pclass no tiene valores nan


# Columna Sex no tiene valores nan
# Se pasan los valores male and female a numericos


# Columna Age tiene valores nan
# Se rellena con valores 0 los nan


# Columna SibSp no tiene valores nan


# Columna Parch no tiene valores nan


# Cabin tiene valores nan y se reemplazan por valores 0


In [231]:
test_df

In [232]:
test_df = sc.transform(test_df)

In [233]:
test_df

In [241]:
test_df_tensor = torch.tensor(data=test_df, dtype=torch.float32)

In [275]:
titanic_model.eval()
with torch.no_grad():
    predictions = titanic_model(test_df_tensor)

In [276]:
predictions = (predictions > 0.5).int()
predictions

In [277]:
result = pd.DataFrame({
    'PassengerId': test_csv['PassengerId'],
    'Survived': predictions.numpy().flatten()
})

In [278]:
result.to_csv('files/titanic_predictions.csv', index=False)

In [279]:
result